In [83]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# print(train.info())
# print(test.info())

# 결측치 처리 - 'Item_Weight', 'Outlet_Size'
min = train['Item_Weight'].min()
# print(min)
train['Item_Weight'] = train['Item_Weight'].fillna(min)
test['Item_Weight'] = test['Item_Weight'].fillna(min)

mode = train['Outlet_Size'].mode()[0]
# print(mode)
train['Outlet_Size'] = train['Outlet_Size'].fillna(mode)
test['Outlet_Size'] = test['Outlet_Size'].fillna(mode)

# print(train.isnull().sum())

# 원-핫 인코딩 - 문자형 -> 숫자형(0, 1)
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)
# print(train.shape, test.shape)
# print(train.info())
# print(test.info())

# 불필요한 칼럼 삭제
train.drop('Item_Identifier', axis=1, inplace=True)
test.drop('Item_Identifier', axis=1, inplace=True)

# print(train.info())

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()  # 객체 생성
# cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 
#         'Outlet_Location_Type', 'Outlet_Type']
cols = train.select_dtypes(include="object").columns
print(cols)

for col in cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.fit_transform(test[col])

# 원-핫 인코딩
"""
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)
"""

# 데이터 분할
from sklearn.model_selection import train_test_split
# 데이터 선택
X = train.drop(columns=['Item_Outlet_Sales'])
y = train['Item_Outlet_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
print(X_train.shape,  X_test.shape, y_train.shape, y_test.shape)

# 회귀 모델 학습 및 예측
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=11)
model.fit(X_train, y_train)
pred = model.predict(X_test)
# print(pred)

# 평가지표에 의한 성능 평가
from sklearn.metrics import mean_squared_error
# 평균 제곱 오차
mse = mean_squared_error(y_test, pred)
print('MSE:', mse)

# 평균 제곱 오차
def rmse(y_test, pred):
    return mean_squared_error(y_test, pred) ** 0.5

rmse = rmse(y_test, pred)
print("RMSE:", rmse)

# 평가용 데이터 예측
pred2 = model.predict(test)
print(pred2)

# 파일 제출
df = pd.DataFrame({
    "pred": pred2
})
df.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')
(5454, 10) (1364, 10) (5454,) (1364,)
MSE: 1233834.6661155766
RMSE: 1110.7811063011366
[1556.999932  713.045168 2152.178526 ... 4112.253778  970.842928
 2024.99741 ]


,pred
0,1556.999932
1,713.045168
2,2152.178526
3,1239.732916
4,2937.922396
...,...
1700,284.742686
1701,693.310856
1702,4112.253778
1703,970.842928
